all imports

In [ ]:
import pandas as pd
import re
import string
import matplotlib.pyplot as plt
import spacy
import numpy as np


LOAD DATASET

In [88]:
# Code for 1 database 
df1 = pd.read_csv("data/emails.csv", encoding='latin-1')
df1.columns = ['text', 'label']
df1['label'] = df1['label'].replace([1],'spam')
df1['label'] = df1['label'].replace([0],'ham')

# Code for 2 database
df2 = pd.read_csv("data/spam_ham_dataset.csv", encoding='latin-1')
df2.drop(['label_num', 'Unnamed: 0'], axis=1, inplace=True)
df2 = df2[['text', 'label']]

# Code for 3 database
df3 = pd.read_csv("data/spam.csv", encoding='latin-1')
df3.dropna(how="any", inplace=True, axis=1)
df3.columns = ['label', 'text']
df3 = df3[['text', 'label']]

# Calculate text
df1['text_length'] = df1['text'].apply(lambda x: len(x))
df2['text_length'] = df2['text'].apply(lambda x: len(x))
df3['text_length'] = df3['text'].apply(lambda x: len(x))

PREPROCESSING DANYCH

In [89]:
#first method
from gensim.utils import simple_preprocess
df1['tokens_pp1'] = df1['text'].apply(lambda x: simple_preprocess(x, min_len=2, max_len=15))
df2['tokens_pp1'] = df2['text'].apply(lambda x: simple_preprocess(x, min_len=2, max_len=15))
df3['tokens_pp1'] = df3['text'].apply(lambda x: simple_preprocess(x, min_len=2, max_len=15))

In [ ]:
#second method
nlp = spacy.load("en_core_web_sm")
nlp.disable_pipe("ner")
nlp.disable_pipe("tok2vec")
def spacy_tokenizer(document):
    tokens = nlp(document)
    tokens = [token.lemma_ for token in tokens if (
        token.is_stop == False and \
        token.is_punct == False and \
        token.lemma_.strip()!= '')]
    return tokens

df1['tokens_pp2'] = df1['text'].apply(spacy_tokenizer)
print("done1")
df2['tokens_pp2'] = df2['text'].apply(spacy_tokenizer)
print("done2")
df3['tokens_pp2'] = df3['text'].apply(spacy_tokenizer)


In [91]:
#third method
def remove_punctuation(text):
    no_punct="".join([words for words in text if words not in string.punctuation])
    return no_punct

from nltk.corpus import stopwords
stopword = set(stopwords.words('english'))
stopword.add('Subject')
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stopword])

def tokenize(text):
    split=re.split("\W+",text) 
    return split

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
df1['tokens_pp3'] = df1['text'].apply(remove_punctuation)
df1['tokens_pp3'] = df1['tokens_pp3'].apply(remove_stopwords)
df1['tokens_pp3'] = df1['tokens_pp3'].apply(tokenize)
df1['tokens_pp3'] = df1['tokens_pp3'].apply(lambda x: [stemmer.stem(y) for y in x])

df2['tokens_pp3'] = df2['text'].apply(remove_punctuation)
df2['tokens_pp3'] = df2['tokens_pp3'].apply(remove_stopwords)
df2['tokens_pp3'] = df2['tokens_pp3'].apply(tokenize)
df2['tokens_pp3'] = df2['tokens_pp3'].apply(lambda x: [stemmer.stem(y) for y in x])

df3['tokens_pp3'] = df3['text'].apply(remove_punctuation)
df3['tokens_pp3'] = df3['tokens_pp3'].apply(remove_stopwords)
df3['tokens_pp3'] = df3['tokens_pp3'].apply(tokenize)
df3['tokens_pp3'] = df3['tokens_pp3'].apply(lambda x: [stemmer.stem(y) for y in x])

FEATURE EXTRACTION

In [ ]:
#first method TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

n_pca_features = 100

tfidf_result = []
i = 0
# for df in [df1,df2,df3]:
for df in [df3]:
  for x in [df["tokens_pp1"],df["tokens_pp2"],df["tokens_pp3"]]:
      print("Iteration number",i)
      tfidf_vector = TfidfVectorizer(input="content", analyzer="word")
      result = tfidf_vector.fit_transform(x.apply(lambda x: ' '.join(x)))
      dense = result.todense()
      denselist = dense.tolist()
      df_proc = pd.DataFrame(denselist,columns=tfidf_vector.get_feature_names_out())
      scalar = StandardScaler()
      scalar.fit(df_proc)
      scaled_data = scalar.transform(df_proc)
      pca = PCA(n_components = n_pca_features)
      pca.fit(scaled_data)
      x_pca = pca.transform(scaled_data)
      tmp = np.array(df['text_length'])
      features = np.concatenate((x_pca, tmp[:,np.newaxis]), axis=1)
      tfidf_result.append(features)
      i+=1


In [ ]:
# second method Bag of words
from sklearn.feature_extraction.text import CountVectorizer

def dummy(doc):
  return doc

bow_result = []
i = 0
for df in [df1, df2, df3]:
  for x in [df["tokens_pp1"],df["tokens_pp2"],df["tokens_pp3"]]:
    print(f"Iteration number: {i}")
    cv = CountVectorizer(tokenizer=dummy, preprocessor=dummy,)
    counts = cv.fit_transform(x)
    print("check")
    dense = counts.todense()
    denselist = dense.tolist()
    df_proc = pd.DataFrame(denselist,columns=cv.get_feature_names())
    scalar = StandardScaler()
    scalar.fit(df_proc)
    scaled_data = scalar.transform(df_proc)
    pca = PCA(n_components = n_pca_features)
    pca.fit(scaled_data)
    x_pca = pca.transform(scaled_data)
    tmp = np.array(df['text_length'])
    features = np.concatenate((x_pca, tmp[:,np.newaxis]), axis=1)
    bow_result.append(features)
    i+=1


In [162]:
# third method word2vec
from gensim.models import Word2Vec

n_pca_features = 20
vector_size=100
def word2vec(tokens):
    if len(tokens) > 0:
        word2vec = model.wv[tokens]
        return np.average(word2vec, axis=0)
    else:
        # When there is no token in tokens
        return np.zeros(vector_size)

word2vec_result = []
df1 = df1.sample(frac=1).reset_index(drop=True)
dfSUM = pd.concat([df1.iloc[1000:,:], df3], axis=0,ignore_index=True)
for df in [dfSUM]:
  for x in [df["tokens_pp3"]]:
    model = Word2Vec(
        window=5,
        min_count=1,
        workers=4,
        vector_size=vector_size
    )
    model.build_vocab(x)
    model.train(x, total_examples=model.corpus_count, epochs=100)
    
    result = x.apply(word2vec)
    pca = PCA(n_components=n_pca_features)
    x_pca = pca.fit_transform(pd.DataFrame.from_records(result))
    tmp = np.array(df['text_length'])
    features = np.concatenate((x_pca, tmp[:,np.newaxis]), axis=1)
    word2vec_result.append(features)
    

PĘTLA EKSPERYMENTALNA

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import accuracy_score

i = 0
metrics_of_tfidf = []
# for df in [df1, df2, df3]:
for df in [df3]:
    y = np.array(df['label'])
    # print(y)
    crossvalid_scores = []
    for X in tfidf_result[3*i:3*i+3]:
        rkf = RepeatedKFold(n_splits=5, n_repeats=2,random_state=1234)
        accuracies = []
        for train_index, test_index in rkf.split(X):
            # print("TRAIN:", train_index, "TEST:", test_index)
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            gnb = GaussianNB()
            y_pred = gnb.fit(X_train, y_train).predict(X_test)
            accuracies.append(accuracy_score(y_pred, y_test))
        crossvalid_scores.append(accuracies)
    metrics_of_tfidf.append(crossvalid_scores)    
    i+=1

print((np.shape(metrics_of_tfidf)))
print(((metrics_of_tfidf)))




In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import accuracy_score

i = 0
metrics_of_bagofwords = []
for df in [df1, df2, df3]:
    y = df['label']
    # print(y)
    crossvalid_scores = []
    for X in tfidf_result[3*i:3*i+3]:
        rkf = RepeatedKFold(n_splits=5, n_repeats=2,random_state=1234)
        accuracies = []
        for train_index, test_index in rkf.split(X):
            # print("TRAIN:", train_index, "TEST:", test_index)
            X_train, X_test = X[train_index], y[test_index]
            y_train, y_test = X[train_index], y[test_index]
            gnb = GaussianNB()
            y_pred = gnb.fit(X_train, y_train).predict(X_test)
            accuracies.append(accuracy_score(y_pred, y_test))
        crossvalid_scores.append(accuracies)
    metrics_of_bagofwords.append(crossvalid_scores)    
    i+=1

print((np.shape(metrics_of_bagofwords)))


In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import accuracy_score

i = 0
metrics_of_word2vec = []
for df in [df1]:
    y = np.array(df['label'])
    # print(y)
    crossvalid_scores = []
    for X in word2vec_result[3*i:3*i+3]:
        # print(X)
        rkf = RepeatedKFold(n_splits=5, n_repeats=2,random_state=1234)
        accuracies = []
        for train_index, test_index in rkf.split(X):
            # print("TRAIN:", train_index, "TEST:", test_index)
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            gnb = GaussianNB()
            y_pred = gnb.fit(X_train, y_train).predict(X_test)
            accuracies.append(accuracy_score(y_pred, y_test))
        crossvalid_scores.append(accuracies)
    metrics_of_word2vec.append(crossvalid_scores)    
    i+=1


print((np.shape(metrics_of_word2vec)))
print(((metrics_of_word2vec)))
    

TEST D1 NA WYTRENOWANYM D3

In [ ]:
for df in [df3]:
    y = np.array(df['label'])
    for X in word2vec_result:
      gnb = GaussianNB()
      gnb.fit(X, y)
      
#test
for df in [df1]:
  for x in [df["tokens_pp3"]]:
    model.build_vocab(x,update=True)
    result = x.apply(word2vec)
    pca = PCA(n_components=n_pca_features)
    x_pca = pca.fit_transform(pd.DataFrame.from_records(result))
    tmp = np.array(df['text_length'])
    features = np.concatenate((x_pca, tmp[:,np.newaxis]), axis=1)
    switch_test = features

from sklearn.metrics import classification_report
for df in [df1]:
    y = np.array(df['label'])
    X = switch_test
    y_pred = gnb.predict(X)
    print(accuracy_score(y_pred,y))
    print(classification_report(y, y_pred))

TEST D3 NA D1

In [ ]:
for df in [df2]:
    y = np.array(df['label'])
    for X in word2vec_result:
      gnb = GaussianNB()
      gnb.fit(X, y)
      
#test
for df in [df3]:
  for x in [df["tokens_pp3"]]:
    model.build_vocab(x,update=True)
    result = x.apply(word2vec)
    pca = PCA(n_components=n_pca_features)
    x_pca = pca.fit_transform(pd.DataFrame.from_records(result))
    tmp = np.array(df['text_length'])
    features = np.concatenate((x_pca, tmp[:,np.newaxis]), axis=1)
    switch_test = features

from sklearn.metrics import classification_report
for df in [df3]:
    y = np.array(df['label'])
    X = switch_test
    y_pred = gnb.predict(X)
    print(accuracy_score(y_pred,y))
    print(classification_report(y, y_pred))

TEST MIX

In [ ]:
for df in [dfSUM]:
    # print(df)
    y = np.array(df['label'])
    for X in word2vec_result:
      gnb = GaussianNB()
      print(y)
      gnb.fit(X, y)
      
#test

for df in [df1.iloc[:1000,:]]:
  for x in [df["tokens_pp3"]]:
    model.build_vocab(x,update=True)
    result = x.apply(word2vec)
    pca = PCA(n_components=n_pca_features)
    x_pca = pca.fit_transform(pd.DataFrame.from_records(result))
    tmp = np.array(df['text_length'])
    features = np.concatenate((x_pca, tmp[:,np.newaxis]), axis=1)
    switch_test = features

from sklearn.metrics import classification_report
for df in [df1.iloc[:1000,:]]:
    y = np.array(df['label'])
    X = switch_test
    y_pred = gnb.predict(X)
    print(accuracy_score(y_pred,y))
    print(classification_report(y, y_pred))

In [ ]:
from scipy.stats import ttest_rel

alfa = .05
n_items = 9
t_statistic = np.zeros((n_items,n_items))
p_value = np.zeros((n_items,n_items))

# scores_d1 = [*metrics_of_tfidf[0], *metrics_of_bagofwords[0], *metrics_of_word2vec[0]]
# scores_d2 = [*metrics_of_tfidf[1], *metrics_of_bagofwords[1], *metrics_of_word2vec[1]]
# scores_d3 = [*metrics_of_tfidf[2], *metrics_of_bagofwords[2], *metrics_of_word2vec[2]]
scores_d1 = [
    *[[0.7574171029668412, 0.7513089005235603, 0.7696335078534031, 0.7694323144104803, 0.7720524017467248, 0.7591623036649214, 0.27137870855148344, 0.7713787085514834, 0.7563318777292577, 0.765938864628821], [0.7705061082024433, 0.7504363001745201, 0.7766143106457243, 0.7703056768558952, 0.7816593886462883, 0.7591623036649214, 0.7731239092495636, 0.7757417102966842, 0.7580786026200873, 0.7720524017467248], [0.768760907504363, 0.7591623036649214, 0.7705061082024433, 0.7650655021834061, 0.7039301310043669, 0.7556719022687609, 0.7801047120418848, 0.7766143106457243, 0.7615720524017467, 0.7554585152838428]],
    *[[0.8708551483420593, 0.8621291448516579, 0.87521815008726, 0.8558951965065502, 0.8655021834061135, 0.8542757417102966, 0.8717277486910995, 0.8787085514834206, 0.8655021834061135, 0.8541484716157205], [0.9048865619546248, 0.918848167539267, 0.9179755671902269, 0.8995633187772926, 0.9039301310043668, 0.8996509598603839, 0.9179755671902269, 0.9153577661431065, 0.9126637554585153, 0.9082969432314411], [0.8996509598603839, 0.8979057591623036, 0.912739965095986, 0.8873362445414847, 0.8873362445414847, 0.887434554973822, 0.9083769633507853, 0.9040139616055847, 0.891703056768559, 0.8960698689956332]],
    *[[0.7626527050610821, 0.7556719022687609, 0.7670157068062827, 0.7668122270742358, 0.7746724890829695, 0.7556719022687609, 0.7722513089005235, 0.7705061082024433, 0.7510917030567685, 0.7729257641921398], [0.7609075043630017, 0.7547993019197208, 0.7731239092495636, 0.7685589519650655, 0.7746724890829695, 0.7591623036649214, 0.7722513089005235, 0.7731239092495636, 0.7572052401746725, 0.77117903930131], [0.7556719022687609, 0.7102966841186736, 0.7722513089005235, 0.7580786026200873, 0.7737991266375546, 0.7556719022687609, 0.7661431064572426, 0.7696335078534031, 0.7554585152838428, 0.7668122270742358]]
]
scores_d2 = [
    *[[0.7391304347826086, 0.7659574468085106, 0.7543520309477756, 0.7340425531914894, 0.7214700193423598, 0.7352657004830918, 0.7398452611218569, 0.7601547388781431, 0.7330754352030948, 0.7437137330754352], [0.7381642512077294, 0.7620889748549323, 0.758220502901354, 0.7311411992263056, 0.7214700193423598, 0.740096618357488, 0.7398452611218569, 0.7572533849129593, 0.7292069632495164, 0.741779497098646], [0.7439613526570048, 0.7678916827852998, 0.7543520309477756, 0.7330754352030948, 0.7272727272727273, 0.7333333333333333, 0.7446808510638298, 0.7620889748549323, 0.7408123791102514, 0.7437137330754352]],
    *[[0.9265700483091788, 0.9313346228239845, 0.925531914893617, 0.9294003868471954, 0.9081237911025145, 0.9246376811594202, 0.9177949709864603, 0.9342359767891683, 0.9235976789168279, 0.9081237911025145], [0.9207729468599034, 0.9235976789168279, 0.9235976789168279, 0.9497098646034816, 0.9352030947775629, 0.9468599033816425, 0.913926499032882, 0.9506769825918762, 0.9313346228239845, 0.9168278529980658], [0.9314009661835749, 0.9235976789168279, 0.9284332688588007, 0.9448742746615088, 0.9439071566731141, 0.9439613526570049, 0.9177949709864603, 0.9516441005802708, 0.9323017408123792, 0.9216634429400387]],
    *[[0.7516908212560387, 0.7678916827852998, 0.758220502901354, 0.7330754352030948, 0.7388781431334622, 0.7458937198067633, 0.7475822050290135, 0.7630560928433269, 0.7427466150870407, 0.7495164410058027], [0.744927536231884, 0.7611218568665378, 0.7601547388781431, 0.730174081237911, 0.7359767891682786, 0.7478260869565218, 0.7456479690522244, 0.7591876208897486, 0.7350096711798839, 0.7485493230174082], [0.7391304347826086, 0.758220502901354, 0.7524177949709865, 0.7253384912959381, 0.723404255319149, 0.7391304347826086, 0.7379110251450677, 0.7562862669245648, 0.7330754352030948, 0.7427466150870407]]
]
scores_d3 = [
    *[[0.20269058295964126, 0.17399103139013453, 0.19658886894075403, 0.20287253141831238, 0.18312387791741472, 0.21614349775784752, 0.18744394618834082, 0.19389587073608616, 0.1741472172351885, 0.19210053859964094], [0.18475336322869956, 0.14887892376681613, 0.21364452423698385, 0.17594254937163376, 0.1732495511669659, 0.6026905829596413, 0.15874439461883408, 0.1822262118491921, 0.16427289048473967, 0.16786355475763015], [0.17847533632286997, 0.14349775784753363, 0.16427289048473967, 0.16337522441651706, 0.15978456014362658, 0.4439461883408072, 0.15336322869955157, 0.17145421903052063, 0.1526032315978456, 0.15978456014362658]],
    *[[0.9865470852017937, 0.979372197309417, 0.9829443447037702, 0.9793536804308797, 0.9820466786355476, 0.9838565022421525, 0.9865470852017937, 0.9784560143626571, 0.9829443447037702, 0.9793536804308797], [0.9748878923766816, 0.9757847533632287, 0.9784560143626571, 0.9766606822262118, 0.9784560143626571, 0.9730941704035875, 0.9757847533632287, 0.9766606822262118, 0.9802513464991023, 0.9820466786355476], [0.9766816143497757, 0.9739910313901345, 0.9847396768402155, 0.9802513464991023, 0.9775583482944344, 0.9748878923766816, 0.9739910313901345, 0.9766606822262118, 0.9847396768402155, 0.9793536804308797]],
    *[[0.20448430493273542, 0.17668161434977578, 0.2001795332136445, 0.20466786355475763, 0.18940754039497307, 0.21524663677130046, 0.19103139013452916, 0.20915619389587073, 0.17863554757630162, 0.19030520646319568], [0.18923766816143497, 0.16412556053811658, 0.1867145421903052, 0.1867145421903052, 0.16786355475763015, 0.21076233183856502, 0.18026905829596412, 0.19030520646319568, 0.1561938958707361, 0.17145421903052063], [0.19641255605381167, 0.16233183856502242, 0.38420107719928187, 0.1813285457809695, 0.1947935368043088, 0.4591928251121076, 0.18026905829596412, 0.1947935368043088, 0.15888689407540396, 0.17953321364452424]]
]


scr = scores_d3
np.set_printoptions(precision=2)
np.set_printoptions(suppress=True)
np.set_printoptions(edgeitems=30,linewidth=100000)
for i in range(n_items):
    for j in range(n_items):
        t_statistic[i, j], p_value[i, j] = ttest_rel(scr[i], scr[j])
print("t-statistic:\n", t_statistic, "\n\np-value:\n", p_value)

from tabulate import tabulate
headers = ["P1_TF","P2_TF","P3_TF","P1_W2V","P2_W2V","P3_W2V","P1_BOW","P2_BOW","P3_BOW"]
names_column = np.array([["P1_TF"],["P2_TF"],["P3_TF"],["P1_W2V"],["P2_W2V"],["P3_W2V"],["P1_BOW"],["P2_BOW"],["P3_BOW"]])
advantage = np.zeros((9,9))
advantage[t_statistic > 0] = 1
advantage_table = tabulate(np.concatenate(
    (names_column, advantage), axis=1), headers)
# print("Advantage:\n", advantage_table)
significance = np.zeros((9,9))
significance[p_value <= alfa] = 1
significance_table = tabulate(np.concatenate(
    (names_column, significance), axis=1), headers)
# print("Statistical significance (alpha = 0.05):\n", significance_table)
stat_better = significance * advantage
stat_better_table = tabulate(np.concatenate(
    (names_column, stat_better), axis=1), headers)
print("Statistically significantly better:\n", stat_better_table)
i = 0
for s in scr:
    print(headers[i]+' -> {:.2f} ({:.2f})'.format(np.mean(s),np.std(s)))
    i +=1